## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *
import tensorflow_addons as tfa

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
%%time
# 약 20초 걸림
with open('D:/inputs/24/train.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)


with open('D:/inputs/24/test.pickle', 'rb') as f:
    test = pickle.load(f)

# 아웃라이어 수정
train[1,16, 24, -2] = 163.05731201

## augmentation 불러오기 및 합치기

In [ ]:
with open('D:/inputs/24/rot_train180.pickle', 'rb') as f:
    rot_train180 = pickle.load(f)
    
with open('D:/inputs/24/rot_train180_y.pickle', 'rb') as f:
    rot_train180_y = pickle.load(f)
    rot_train180_y = rot_train180_y.reshape(rot_train90y.shape[0], 40, 40, 1)

train = np.vstack([train, rot_train180])
train_y = np.vstack([train_y, rot_train180_y])

In [ ]:
with open('D:/inputs/24/rot_train270.pickle', 'rb') as f:
    rot_train270 = pickle.load(f)
    
with open('D:/inputs/24/rot_train270_y.pickle', 'rb') as f:
    rot_train270y = pickle.load(f)
    rot_train270y = rot_train270y.reshape(rot_train270y.shape[0], 40, 40, 1)

train = np.vstack([train, rot_train270])
train_y = np.vstack([train_y, rot_train270y])

In [ ]:
display(
    train.shape,
    train_y.shape,
    test.shape
)

In [ ]:
# 조건에 맞는 사진 개수 세는 알고리즘.
# from tqdm import tqdm
# cnt = 0
# for i in tqdm(train_y):
#     for ii in i:
#         if (ii == 0).sum() != 40:
#             cnt += 1
# cnt

## all_np로 결합 후 회전변환 피쳐 생성

In [ ]:
# 약 3분 걸림
from features import rotation

train, test = rotation(train, test)

## 9번 channel 가공 - 카테고리형 피쳐

In [ ]:
# [  0., 100., 101., 102., 103., 105., 106., 107., 108., 109., 110.,
#        111., 112., 113., 114., 115., 117., 118., 119., 121., 122., 123.,
#        199., 200., 201., 202., 203., 205., 206., 207., 208., 210., 211.,
#        212., 213., 214., 215., 217., 218., 219., 221., 222., 223., 299.,
#        300., 301., 302., 303., 305., 306., 307., 308., 310., 311., 312.,
#        313., 314., 315., 317., 318., 319., 321., 322.]
# train = np.delete(train, 9, axis=3)
# test = np.delete(test, 9, axis=3)

In [ ]:
from features import terrain

train, test = terrain(train, test)

## all_np로 결합 후 scaling

In [ ]:
## ss 약 5분 걸림
from preprocessing import scaling

train, test = scaling(train, test, 'ss') # 3번째 인자 'rs', 'ms', 'ss' 


In [ ]:
%%time
np.save('D:/inputs/24/ss_train_rotsum.npy', train)
np.save('D:/inputs/24/ss_test_rotsum.npy', test)

In [ ]:
gc.collect()

## scaling 파일 불러오기

In [ ]:
%%time
train = np.load('D:/inputs/24/ss_train_rotsum.npy')
test = np.load('D:/inputs/24/ss_test_rotsum.npy')


In [ ]:
with open('D:/inputs/24/train_y.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)

In [ ]:
from models import build_model, build_model2
from tensorflow.keras.layers import Input
from tensorflow.keras import Model

input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model(input_layer, 32)
model = Model(input_layer, output_layer)

# sgd = tf.keras.optimizers.SGD(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(sgd)

# ad = tf.keras.optimizers.Adam(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(ad)
# model.compile(loss="mae", optimizer=stocastic_avg_opt, metrics=["mae"])
#


# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])
model.compile(loss="mae", optimizer="adam", metrics=["mae"])

scores = []

In [ ]:
model = tf.keras.models.load_model('epochs_8.h5')
model.summary()

In [ ]:
EPOCHS = 30
for _ in range(5):
    for i in range(EPOCHS):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 200)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
plt.plot(scores)

In [ ]:
EPOCHS = 30
for _ in range(5):
    for i in range(EPOCHS):
        print(f"epochs: {i+1}")
        model_history = model.fit(train, train_y, epochs = 1, verbose=1, batch_size = 200)
    score = maeOverFscore(train_y, model.predict(train))
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)

## 교차 검증

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(train, train_y, shuffle=True, random_state=0):
    X_train, X_test = train[train_index], train[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]

## mae loss graph

In [ ]:
scores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
plt.plot(scores)

## 모델 저장 및 submission 생성

In [ ]:
model.save('rot480.h5')

In [ ]:
model.save('epochs.h5')

In [ ]:
pred = model.predict(test)
submission = pd.read_csv('inputs/sample_submission.csv')
submission.iloc[:,1:] = pred.reshape(-1, 1600)
submission.to_csv('nodropout_3.csv', index = False)

## 해야할 일

1. ~~스케일링(Scaling) - Robust, log, Standard Scaler  ~~  
스케일링 방법을 다르게 해야하나. 하나의 사진을 기준으로 처리해야 할지 아니면 1번픽셀, 2번픽셀... 1600픽셀 끼리 해야하나
2. ~~위도 경도 피쳐를 가지고 위도+경도, 위도-경도 피쳐 만들기(45도 회전변환)~~: 30, 60도 변환 추가
3. 현재 custom metric이 작동하지 않으므로, 이 또한 수정해야함.
4. tree모델 만들고, 앙상블
5. ~~target 값에 log 취하기~~: 점수 안오름
6. 순서가 바뀌긴 했으나, EDA 작업하자.
7. 강수량 타일이 10개 미만인 것의 평균, 20개미만의 평균.... 50개 미만의 평균... 알아보자
8. ~~augumentation 사용해서 강수량이 일정수준 이상인 사진 augementation 하자~~ - 다만, regression인 만큼 가장자리 어떻게 처리할지 생각해보자. - 정사각형 사진이기에 90도 돌린사진, 180도 돌린사진, 270도 돌린사진 이렇게만 해도 데이터가 증식될거같음 -  
**그런데!!!!!** 생각해봐야할것이.....  
위도, 경도 등 여러 요소가 존재할텐데 돌려버리면 원래는 비 안오는거를 비온다고 볼 수도 있을 듯 싶다. 쉽게 예를 들면 I 모양일때는 비가 오고, ㅡ 모양일때는 비가안오는데, 돌리는 바람에 안오는걸 온다고 할 수도 있을 것 같다. - 이건 테스트해볼수밖에 없을 듯 싶다.  
돌릴수있는방법도 여러가지이다. 90도씩회전하기, 수직축기준으로 전환, 수평축기준으로 전환 등 - 캐글에서는 수직축, 수평축으로 전환해서 augmentation 썼다고 함.  위성이 도는 방향을 보면, 270도 돌리는 것도 괜찮을듯
9. pretrained 모델 활용방안 생각하기 - 최소 사진크기가 존재하는데 우리는 40x40이라 작은편.. 이걸 40x40을 이어 붙여서 200x200으로 만들어서 넣으면 될거같은데 시도해보자.
10. 9번채널 카테고리 변수 원핫인코딩
11. Unet 모델과 FPN 모델 앙상블해서 제출하자
12. loss='mse', metric=['mse', 'mae'] 이렇게 해보자

| 리더보드(5부터) | mae / fscore | mae | 비고 |
|---|:---:|---:|---:|
| 1.93737 | 1.8744 | 0.2451 |  |
| 1.75177 | 1.7016 | 0.2384 |  |
| 1.61793 | 1.4234 | 0.0832 | 리더보드랑 metric 점수랑 차이 많이 남. 해결해야함. |